In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import category_encoders as ce
import functools

from keras.layers import Concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D
# from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import backend as K
# from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher

# Lectura de datos

In [23]:
data = pd.read_csv('D:/Visual Studio Code/HK 6/Data Mining/RASFF-predictions-dashboard/Copia csvs antiguos/splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
df = data
df

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
101816,2010-04-21,Slovenia,"dietetic foods, food supplements, fortified foods",food,undecided,re-dispatch,distribution restricted to notifying country,allergens,Serbia,Slovenia,
104248,2009-12-10,Slovenia,food contact materials,fcm,undecided,public warning - press release,distribution on the market (possible),metals,China,Poland,Czech Republic
99674,2010-08-11,Netherlands,"nuts, nut products and seeds",food,undecided,official detention,no distribution,mycotoxins,Argentina,,Spain
100034,2010-07-28,United Kingdom,herbs and spices,food,undecided,(obsolete),no distribution,mycotoxins,India,,United Kingdom
136057,2005-02-18,United Kingdom,"soups, broths, sauces and condiments",food,undecided,(obsolete),distribution on the market (possible),composition,United Kingdom,Russia,Sweden
...,...,...,...,...,...,...,...,...,...,...,...
32626,2017-07-20,Belgium,eggs and egg products,food,serious,withdrawal from the market,distribution to other member countries,pesticide residues,,Congo,Ukraine
141375,2004-01-12,Netherlands,fruits and vegetables,food,undecided,re-dispatch,,mycotoxins,Turkey,,Netherlands
2965,2019-06-20,Denmark,food additives and flavourings,food,undecided,withdrawal from recipient(s),distribution to other member countries,foreign bodies,Belgium,Czech Republic,INFOSAN
48591,2016-02-01,Netherlands,"dietetic foods, food supplements, fortified foods",food,undecided,withdrawal from the market,distribution to other member countries,environmental pollutants,United States,Netherlands,Commission Services


In [8]:
mask = (df['DATE_CASE'] >= '2004-01-01') & (df['DATE_CASE'] <= '2018-12-31')
df = df.loc[mask]
df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
df['DATE_CASE'] = df.DATE_CASE.dt.month

C:\Users\Locc\AppData\Local\Temp\ipykernel_5580\335135510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
C:\Users\Locc\AppData\Local\Temp\ipykernel_5580\335135510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] = df.DATE_CASE.dt.month


In [9]:
features = [0,1,2,6,7,8]
target = [5]
X = df.iloc[:,features]
y = df.iloc[:,target]

In [10]:
X

,DATE_CASE,NOT_COUNTRY,PROD_CAT,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN
97237,12,Greece,cereals and bakery products,distribution on the market (possible),food additives and flavourings,Greece
139207,7,Denmark,herbs and spices,,composition,India
123875,2,Spain,bivalve molluscs and products thereof,no distribution,adulteration / fraud,China
55331,5,France,milk and milk products,distribution to other member countries,pathogenic micro-organisms,France
36508,4,Poland,"cocoa and cocoa preparations, coffee and tea",product not (yet) placed on the market,pesticide residues,India
...,...,...,...,...,...,...
22755,3,United Kingdom,milk and milk products,distribution to non-member countries,foreign bodies,United Kingdom
90314,10,Finland,fruits and vegetables,information on distribution not (yet) available,food additives and flavourings,Philippines
128575,4,Italy,crustaceans and products thereof,distribution on the market (possible),food additives and flavourings,Spain
15217,8,Switzerland,"dietetic foods, food supplements, fortified foods",distribution to other member countries,composition,Slovenia


In [11]:
y

,ACTION_TAKEN
97237,recall from consumers
139207,destruction
123875,destruction
55331,recall from consumers
36508,re-dispatch
...,...
22755,recall from consumers
90314,
128575,
15217,withdrawal from the market


# Selección de codificación

In [14]:
ency = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
ency.fit(y.values)
y_one_hot = ency.transform(y.values)

In [15]:
'''encx = OrdinalEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

'encx = OrdinalEncoder()\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)'

In [16]:
'''encx = OneHotEncoder(handle_unknown='ignore', sparse = False)
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

"encx = OneHotEncoder(handle_unknown='ignore', sparse = False)\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)"

In [17]:
encx  = ce.BaseNEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)

c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWa

In [18]:
X_train, X_val, y_train, y_val = train_test_split(x_one_hot, y_one_hot, test_size=0.2)

# Pruebas con mlp básico

In [19]:
accs = []
for i in range (1,6):
    K.clear_session()
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=len(X_train.values[1])))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation = "softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam",  metrics=['categorical_accuracy'])
    hist = model.fit(X_train,y_train, epochs= 15,validation_data=(X_val, y_val), batch_size = 64)
    accs.append(hist.history['val_categorical_accuracy'][9])
    model.save ("model"+str(i)+".h5")


Epoch 1/15


c:\Users\Locc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1647/1647 ━━━━━━━━━━━━━━━━━━━━ 60s 36ms/step - categorical_accuracy: 0.4245 - loss: 1.8945 - val_categorical_accuracy: 0.5836 - val_loss: 1.3475
Epoch 2/15
 499/1647 ━━━━━━━━━━━━━━━━━━━━ 43s 38ms/step - categorical_accuracy: 0.5795 - loss: 1.3417

KeyboardInterrupt: 

In [ ]:
suma = 0
for i in accs:
    suma = suma + i
print(suma/5)

0.7319564144768591
